Basic configurations

# Credentials for Azure SQL database (using Azure Key Vault)

In [0]:
sql_db_retail_key = dbutils.secrets.get(scope="keys", key="sqldbretailkey")
sql_db_retail_url = "jdbc:sqlserver://retail-oltp-server.database.windows.net:1433;databaseName=retail"
sql_db_retail_user = "andre"

spark.conf.set("spark.sql.retail_url", sql_db_retail_url)
spark.conf.set("spark.sql.retail_user", sql_db_retail_user)
spark.conf.set("spark.sql.retail_key", sql_db_retail_key)

# Credentials for mounting (using Azure Key Vault)

In [0]:
# For accessing the storage accounting
storage_account_key= dbutils.secrets.get(scope="keys", key="storageaccountkey")
mount_point = "/mnt/storageforretail/container"

# Time zone

In [0]:
# Configuring my local time zone
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
# Defining the credentials of Azure SQL database

sql_db_retail_key = "@ndr3!1234"



# Storage Accounting Key (defined in Azure Key Vault)

# Mounting containers and directories

In [0]:
mount_point = "/mnt/storageforretail/container"


if dbutils.fs.ls(mount_point):
    print('OK: container')

else:    
    # Mount the container in Databricks
    dbutils.fs.mount(
        source = "wasbs://container@storageforretail.blob.core.windows.net",
        mount_point = mount_point,
        extra_configs = {"fs.azure.account.key.storageforretail.blob.core.windows.net": storage_account_key}
    )
    print('OK: container')

# Create directories
if dbutils.fs.ls(mount_point):
    print('OK: container/bronze')
else:
    dbutils.fs.mkdirs(f"{mount_point}/bronze")
    print('OK: container/bronze')

if dbutils.fs.ls(mount_point):
    print('OK: container/silver')
else:
    dbutils.fs.mkdirs(f"{mount_point}/silver")
    print('OK: container/silver')

if dbutils.fs.ls(mount_point):
    print('OK: container/gold')
else:
    dbutils.fs.mkdirs(f"{mount_point}/gold")
    print('OK: retailcontainer/gold')


# Creating database for bronze, silver and gold

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS bronze LOCATION '/mnt/databricksretail/retailcontainer/bronze'")
spark.sql("CREATE DATABASE IF NOT EXISTS silver LOCATION '/mnt/databricksretail/retailcontainer/silver'")
spark.sql("CREATE DATABASE IF NOT EXISTS gold LOCATION '/mnt/databricksretail/retailcontainer/gold'")

Check the IP address

In [0]:
import requests
ip = requests.get("https://ifconfig.me").text
print(f"O IP público do Databricks é: {ip}")

Check connection to database

In [0]:
%python
# Define the JDBC URL and authentication credentials
properties = {
    "user": sql_db_retail_user,        # Replace with your username
    "password": sql_db_retail_key,      # Replace with your password
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Try to access the table to validate the connection
try:
    df = spark.read.jdbc(url=sql_db_retail_url, table="INFORMATION_SCHEMA.TABLES", properties=properties)
    df.show()  # Displays the tables in the database
except Exception as e:
    print("Connection error:", e)

# Creating tables

## Bronze tables

Declaring the tables

In [0]:
%sql


In [0]:
# Dictionary with tables and their primary keys
primary_keys = {
    "CUSTOMERS": ["CUSTOMER_ID"],
    "BRANDS": ["BRAND_ID"],
    "CATEGORIES": ["CATEGORY_ID"],
    "PRODUCTS": ["PRODUCT_ID"],
    "STORES": ["STORE_ID"],
    "PROMOTIONS": ["PROMOTION_ID"],
    "PAYMENT_METHODS": ["PAYMENT_METHOD_ID"],
    "INVENTORY": ["INVENTORY_ID"],
    "SALES": ["SALE_ID"],
    "TRANSACTION_ITEM": ["TRANSACTION_ID"] 
}

In [0]:


# Creating tables for the first time only
for table in primary_keys:
    # Get the list of primary keys for the current table
    keys = primary_keys[table]

    # Convert the list of primary keys into a SQL column definition
    primary_keys_str = ",\n  ".join([f"{key} INTEGER" for key in keys]) # DEALING WITH COMPOSITE KEYS

    # Drop the table if it exists
    spark.sql(f"DROP TABLE IF EXISTS bronze.{table}")

    # Generate and execute the SQL statement
    spark.sql(f"""
        CREATE OR REPLACE TABLE bronze.{table} (
            {primary_keys_str},
            hash STRING,
            `load_timestamp` TIMESTAMP,
            `end_timestamp` TIMESTAMP,
            state INTEGER
        ) USING DELTA
    """)
    print(f"Table bronze.{table} created successfully!")

In [0]:
# Dictionary with tables and their primary keys
primary_keys = {
    "CUSTOMERS": ["CUSTOMER_ID"]
}

Ingestion on bronze

In [0]:
for table in primary_keys:
    # Get the list of primary keys for the current table
    keys = primary_keys[table]

    # Convert the list of primary keys into a SQL column definition
    primary_keys_str = ",\n  ".join([f"{key} INTEGER" for key in keys]) # DEALING WITH COMPOSITE KEYS (just integers)

    # Build condition for merge
    merge_condition = " AND ".join([f"target.{key} = source.{key}" for key in keys])


    df_source = spark.read \
        .jdbc(
            url=sql_db_retail_url,
            table=f"dbo.{table}",
            properties=properties)

    # Add basics collumns to the source DataFrame:
    df_source = df_source.withColumn("hash", sha2(concat_ws("|", *[col(c) for c in df_source.columns if c != "timestamp"]), 256))
    df_source = df_source.withColumn("load_timestamp", current_timestamp())
    df_source = df_source.withColumn("end_timestamp", lit(None).cast("timestamp"))
    df_source = df_source.withColumn("state", lit(2))
    df_source.createOrReplaceTempView("BRONZE_VIEW")

    df_bronze = DeltaTable.forName(spark, f"bronze.{table}")

    # Merge!!!
    spark.sql(f"""
            MERGE INTO bronze.{table} AS target
            USING BRONZE_VIEW AS source
            ON {merge_condition}
            WHEN MATCHED AND target.hash != source.hash AND target.state IN (2, 4) THEN
                UPDATE SET
                    target.state = 3,
                    target.end_timestamp = current_timestamp()
            WHEN NOT MATCHED BY SOURCE THEN
                UPDATE SET
                    target.state = 1,
                    target.end_timestamp = current_timestamp()
            """
            )

    # Convert DeltaTable to DataFrame and filter out the updated (before) records to write to the bronze table

    # mark the records that were updated (after):
    df_source = df_source.withColumn("state", lit(4))

    # filtering df_source to the records that were updated (after)
    new_records_df_4 = df_source.join(
        df_bronze.toDF().filter(col("state") == 3).select(f"{ ' AND target.'.join(keys) }"),  
        on=f"{ ' AND target.'.join(keys) }",
        how="inner"
    )

    df_source = df_source.withColumn("state", lit(2))

    new_records_df_2 = df_source.join(
        df_bronze.toDF().filter(col("state")  3).select(f"{ ' AND target.'.join(keys) }"),  
        on=f"{ ' AND target.'.join(keys) }",
        how="inner"
    )

    # Write the updated (after) records to the bronze table
    new_records_df.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .saveAsTable(f"bronze.{table}")
    
    print(f"Table bronze.{table} updated successfully!")

In [0]:
from pyspark.sql.functions import current_timestamp, col, concat_ws, sha2, lit
from delta.tables import DeltaTable

In [0]:
for table in primary_keys:
    # Get the list of primary keys for the current table
    keys = primary_keys[table]

    # Convert the list of primary keys into a SQL column definition
    primary_keys_str = ",\n  ".join([f"{key} INTEGER" for key in keys]) # DEALING WITH COMPOSITE KEYS (just integers)
    merge_condition = " AND ".join([f"target.{key} = source.{key}" for key in keys])
    ids = ", ".join([f"{key}" for key in keys])

    # Build condition for merge
    merge_condition = " AND ".join([f"target.{key} = source.{key}" for key in keys])

    # taking the bronze table inside delta lake
    df_bronze = DeltaTable.forName(spark, f"bronze.{table}")

    # reading the source table
    df_source = spark.read \
        .jdbc(
            url=sql_db_retail_url,
            table=f"dbo.{table}",
            properties=properties)

    # Add basics collumns to the source DataFrame:
    df_source = df_source.withColumn("hash", sha2(concat_ws("|", *[col(c) for c in df_source.columns if c != "timestamp"]), 256))
    df_source = df_source.withColumn("load_timestamp", current_timestamp())
    df_source = df_source.withColumn("end_timestamp", lit(None).cast("timestamp"))   

    # Mark 1 for delete and 3 for update (before image) in bronze.table
    df_bronze.alias("target").merge(
        df_source.alias("source"),
        merge_condition
    ).whenMatchedUpdate(
        condition="target.hash != source.hash AND target.state in (2,4)",
        set={
            "state": "3",
            "end_timestamp": current_timestamp()
        }
    ).whenNotMatchedBySourceUpdate(
        set={
            "state": "1",
            "end_timestamp": current_timestamp()
        }
    ).execute()

    # Filter rows of df_bronze wich state is 3 (update before image):
    df_bronze_filtered3 = df_bronze.toDF().filter(col("state") == 3)

    # Now, using the filtered bronze rows to mark new records to state = 4 (updated after  image):
    new_records_df4 = df_source \
        .withColumn("state", lit(4)) \
        .join(
            df_bronze_filtered3.select(f"{ ' AND target.'.join(keys) }"),  
            on=f"{ ' AND target.'.join(keys) }",
            how="inner"
        )

    # Filter all rows that is not delete
    df_bronze_filtered234 = df_bronze.toDF().filter(col("state").isin(2, 3, 4))

    # Now, using the filtered bronze rows to mark new records to state = 2 (insert):
    new_records_df2 = df_source \
        .withColumn("state", lit(2)) \
        .join(
            df_bronze_filtered234.select(f"{ ' AND target.'.join(keys) }"),  
            on=f"{ ' AND target.'.join(keys) }",
            how="left_anti"                                                 
        )
    
    new_records_df = new_records_df2.union(new_records_df4)

    # Write the updated (after) records to the bronze table
    new_records_df.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .saveAsTable(f"bronze.{table}")
    
    print(f"Table bronze.{table} updated successfully!")

In [0]:
%sql
select * from bronze.customers;

In [0]:
%sql
select * from bronze.customers;

Schemas of tables

In [0]:
import json
from pyspark.sql.types import StructType, StructField

In [0]:
# Dictionary with tables and their primary keys
primary_keys = {
    "CUSTOMERS": ["CUSTOMER_ID"]
}

In [0]:
for table in primary_keys:

  # taking the bronze table inside delta lake
  df = DeltaTable.forName(spark, f"bronze.{table}")
  
  # Pegando o schema original da tabela Bronze
  schema = df.toDF().schema
  
  # Removendo o metadata de cada StructField
  updated_fields = []
  for field in schema.fields:
      # Cria um novo StructField sem o campo metadata
      updated_field = StructField(field.name, field.dataType, field.nullable)
      updated_fields.append(updated_field)
  
  # Criando o novo schema sem o metadata
  updated_schema = StructType(updated_fields)
  
  # Convertendo o novo schema para JSON
  schema_json = json.dumps(updated_schema.jsonValue(), indent=2)

  # Caminho no DBFS
  path = f"{mount_point}/bronze/schema_{table}.json"

  # Salvando o JSON
  dbutils.fs.put(path, schema_json, True)

In [0]:
display(json.loads(schema_json))

In [0]:
# Lendo o arquivo JSON
schema_json = dbutils.fs.head(path)
schema_dict = json.loads(schema_json)

# Extraindo os nomes das colunas do schema
expected_columns = [field["name"] for field in schema_dict["fields"]]

print(expected_columns)  # ['col_from_schema1', 'col_from_schema2', ...]

# Silver
* Join
* Schema enforcement
* Data quality

## Customers

In [0]:
%sql
-- Reading just active field (enforcing the schemas)
-- Reading exactly the expected field
-- Treating NULL

CREATE OR REPLACE TABLE silver.customers USING DELTA AS
SELECT 
    CUSTOMER_ID,
    hash,
    load_timestamp,
    end_timestamp,
    Social_Security_Number,
    COALESCE(Name, 'NOT INFORMED') AS Name,
    COALESCE(Email, 'NOT INFORMED') AS Email,
    CAST(COALESCE(REGEXP_REPLACE(Phone, '[^0-9]', ''), '-1') AS BIGINT) AS Phone,
    COALESCE(Address, 'NOT INFORMED') AS Address,
    CAST(REGEXP_REPLACE(COALESCE(Zip_Code, '-1'), '[^0-9]', '') AS INT) AS Zip_Code
FROM bronze.customers
-- WHERE {processing_date} BETWEEN load_timestamp AND end_timestamp
WHERE state in (2, 4)




Antes: criar tabela gold

In [0]:
%sql
select * from silver.customers

In [0]:
%sql
select * from bronze.customers

# Gold

## Customer

* SDC2 Dimension

In [0]:
%sql
CREATE OR REPLACE TABLE gold.customers (
  SK_CUSTOMER BIGINT GENERATED ALWAYS AS IDENTITY,
  CUSTOMER_ID INTEGER,
  ACTIVE BOOLEAN,
  load_timestamp TIMESTAMP,
  end_timestamp TIMESTAMP,
  HASH STRING,
  SOCIAL_SECURITY_NUMBER STRING,
  NAME STRING,
  EMAIL STRING,
  PHONE BIGINT,
  ADDRESS STRING,
  ZIP_CODE INTEGER
)
USING DELTA

In [0]:
%sql
SELECT * FROM gold.customers

In [0]:
# reading silver and gold tables
df_silver = DeltaTable.forName(spark, "silver.Customers").toDF()
df_gold = DeltaTable.forName(spark, "gold.Customers")

# Mark 1 for delete and 3 for update (before image) in silver.table
df_gold.alias("target").merge(
    df_silver.alias("source"),
    "target.CUSTOMER_ID == source.CUSTOMER_ID AND target.active == 1"
).whenMatchedUpdate(
    condition="target.hash != source.hash",
    set={
        "active": "0",
        "end_timestamp": current_timestamp()
    }
).whenNotMatchedBySourceUpdate(
    set={
        "active": "0",
        "end_timestamp": current_timestamp()
    }
).execute()

# Insert Updated records to gold table
df_gold.alias("target").merge(
    df_silver.alias("source"),
    "target.CUSTOMER_ID == source.CUSTOMER_ID AND target.active == 1"
).whenNotMatchedInsert(
    values = {
        "CUSTOMER_ID": "source.CUSTOMER_ID",
        "hash": "source.hash",
        "load_timestamp": "source.load_timestamp",
        "end_timestamp": "source.end_timestamp",
        "active": "1",
        "Social_Security_Number": "source.Social_Security_Number",
        "Name": "source.Name",
        "Email": "source.Email",
        "Phone": "source.Phone",
        "Address": "source.Address",
        "Zip_Code": "source.Zip_Code"
        }
).execute()

In [0]:
# 1. Read from bronze.Customers just the active records
# 2. No need for joins (it is a simple example)
# 3.

In [0]:
%sql
DROP TABLE IF EXISTS bronze.customers

In [0]:
# Listar todas as tabelas no Hive Metastore
tables = [
    "CUSTOMERS", "BRANDS", "CATEGORIES", "PRODUCTS", "STORES",
    "PROMOTIONS", "PAYMENT_METHODS", "INVENTORY", "SALES", "TRANSACTION_ITEM"
]

# Excluir todas as tabelas
for table in tables:
    spark.sql(f"DROP TABLE IF EXISTS bronze.{table}")

In [0]:
# Delete all files and directories


dbutils.fs.unmount("/mnt/databricksretail/retailcontainer")
dbutils.fs.rm("/mnt/databricksretail", recurse=True)

In [0]:
# Excluir o banco de dados, incluindo todas as tabelas (opção CASCADE)
spark.sql("DROP DATABASE bronze CASCADE")
spark.sql("DROP DATABASE silver CASCADE")
spark.sql("DROP DATABASE gold CASCADE")

In [0]:
%sql
Select * from bronze.customers

In [0]:
%sql
Select * from SILVER.customers

In [0]:
%sql
Select * from GOLD.customers

In [0]:
# for gold:

# Deduplicate the source DataFrame
window_spec = Window.partitionBy("CUSTOMER_ID").orderBy(col("load_timestamp").desc())
df_silver_dedup = df_silver.withColumn("row_num", row_number().over(window_spec)).filter(col("row_num") == 1).drop("row_num")